In [3]:
# Instalar haystack directamente sin usar entornos virtuales
!pip install farm-haystack

# Confirmar la instalación mostrando la versión
!pip show farm-haystack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 11.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=b4d564b85215d9678f53d6518b4457abc7dc3fc0c2cb3bd80779b267d1b622dd
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481

In [4]:
# Importar las bibliotecas necesarias
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import BM25Retriever, TransformersReader
from haystack.pipelines import ExtractiveQAPipeline
import logging

In [5]:
# Reducir mensajes de log
logging.basicConfig(level=logging.WARNING)

In [6]:
# 1. Crear almacén de documentos
document_store = InMemoryDocumentStore(use_bm25=True)

In [7]:
# 2. Agregar documentos de ejemplo
docs = [
    {"content": "Miguel de Cervantes escribió Don Quijote de la Mancha. Nació en Alcalá de Henares en 1547."},
    {"content": "Madrid es la capital de España y tiene una población de más de 3 millones de habitantes."},
    {"content": "El español es el segundo idioma más hablado del mundo después del chino mandarín."}
]

In [8]:
# 3. Escribir documentos al almacén
document_store.write_documents(docs)

Updating BM25 representation...: 100%|██████████| 3/3 [00:00<00:00, 15611.55 docs/s]


In [9]:
# 4. Crear pipeline de recuperación y lectura
retriever = BM25Retriever(document_store=document_store)
reader = TransformersReader("mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")
pipe = ExtractiveQAPipeline(reader, retriever)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [10]:
# 5. Hacer una pregunta
pregunta = "¿Quién escribió Don Quijote?"
resultado = pipe.run(query=pregunta)

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [11]:
# 6. Mostrar la respuesta
print(f"Pregunta: {pregunta}")
if resultado["answers"]:
    print(f"Respuesta: {resultado['answers'][0].answer}")
    print(f"Puntuación: {resultado['answers'][0].score:.4f}")
else:
    print("No se encontró respuesta.")


Pregunta: ¿Quién escribió Don Quijote?
Respuesta: Miguel de Cervantes
Puntuación: 0.9735


In [12]:
# Prueba otra pregunta
pregunta = "¿Dónde nació Cervantes?"
resultado = pipe.run(query=pregunta)
print(f"\nPregunta: {pregunta}")
if resultado["answers"]:
    print(f"Respuesta: {resultado['answers'][0].answer}")
    print(f"Puntuación: {resultado['answers'][0].score:.4f}")
else:
    print("No se encontró respuesta.")

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



Pregunta: ¿Dónde nació Cervantes?
Respuesta: Alcalá de Henares
Puntuación: 0.9639
